In [2]:
import bql
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

In [3]:
bq = bql.Service()

In [35]:
index = 'CAC Index'
#get monthly date and save into a list
start_date = '2023-02-28'
end_date = '2024-04-30'

date_rng = pd.date_range(start=start_date, end=end_date, freq='M')
month_end_dates = date_rng.strftime('%Y-%m-%d').tolist()
# get stock members at time t
memb_dic={}

for dates in month_end_dates:
    universe = bq.univ.members(index, dates)
    fields = bq.data.id() 
    request = bql.Request(universe, fields)

    response = bq.execute(request)
    df = response[0].df()
    df_member = df["ID()"].tolist()
    memb_dic[dates] = df_member

print("Done")


Done


In [31]:
buyback_1y={}
buyback_2y={}
buyback_3y={}

for dates in month_end_dates:
    bgns={}
    items={}
    universe = memb_dic[dates]
    for i in range(1,4):
        bgns[f'bgn_{i}_y'] = datetime.strptime(dates, '%Y-%m-%d') - relativedelta(years=i)
        items[f'item_{i}_y'] = bq.data.buyback_programs(announced_date=bq.func.range(bgns[f'bgn_{i}_y'], dates),status="ALL",type="OPEN_MARKET")
        ittem = items[f'item_{i}_y']
        request = bql.Request(universe, ittem)
        response = bq.execute(request)
        df_initial = response[0].df().dropna()       
        df_initial=df_initial.reset_index()
        df = df_initial['ID'].drop_duplicates() # make sure no duplicate when we construct the portfolio
        df = df.tolist()
        if i == 1:
            buyback_1y[dates] = df
        elif i == 2:
            buyback_2y[dates] = df
        else:
            buyback_3y[dates] = df
            
print("Done")

Done


In [32]:
#tranform dictionnary to dataframe
df_memb=pd.DataFrame.from_dict(memb_dic,orient='index').transpose()

df_buyback_1y=pd.DataFrame.from_dict(buyback_1y,orient='index').transpose()
df_buyback_2y=pd.DataFrame.from_dict(buyback_2y,orient='index').transpose()
df_buyback_3y=pd.DataFrame.from_dict(buyback_3y,orient='index').transpose()

In [33]:
df_memb.head()

,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30
0,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1CNHI IM Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity
1,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,1COV GY Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity
2,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,A2A IM Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity
3,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAK SS Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity
4,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AAL LN Equity,AALB NA Equity,AALB NA Equity,AALB NA Equity,AALB NA Equity,AALB NA Equity


In [34]:
#export to excel

file_path = 'ptf_CAC.xlsx'  

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    df_memb.to_excel(writer, sheet_name='Members')
    df_buyback_1y.to_excel(writer, sheet_name='Buyback_1y')
    df_buyback_2y.to_excel(writer, sheet_name='Buyback_2y')
    df_buyback_3y.to_excel(writer, sheet_name='Buyback_3y')
